In [1]:
# Import necessary libraries
import pandas as pd
from datetime import datetime

In [2]:
# Read the CSV files into DataFrames
company_df = pd.read_csv('../Data/company_info.csv')
stock_df = pd.read_csv('../Data/company_stock_details.csv')

print(f"Company data shape: {company_df.shape}")
print(f"Stock data shape: {stock_df.shape}")

Company data shape: (524, 4)
Stock data shape: (217811, 18)


In [3]:
missing_company = company_df.isnull().any(axis=1).sum()
print(f"Number of rows in company with missing data: {missing_company}")

missing_stock = stock_df.isnull().any(axis=1).sum()
print(f"Number of rows in stock with missing data: {missing_stock}")

# Since the number of rows with missing data is small, we can drop them
stock_df = stock_df.dropna()

# confirming the number of rows after dropping missing data
print(f"Stock data shape after dropping missing rows: {stock_df.shape}")

Number of rows in company with missing data: 0
Number of rows in stock with missing data: 493
Stock data shape after dropping missing rows: (217318, 18)


In [4]:
# Count the number of unique companies with the frequencies of their stock data
print(stock_df['Symbol'].value_counts())
# Count the number of unique companies in the company DataFrame
print(f"Number of unique companies in company DataFrame: {company_df['Symbol'].nunique()}")

Symbol
MMM     440
NRG     440
NLOK    440
NOC     440
NTRS    440
       ... 
EMN     440
DXC     440
DD      440
OGN     285
CEG     113
Name: count, Length: 495, dtype: int64
Number of unique companies in company DataFrame: 497


In [5]:
# Let's merge the two DataFrames on the 'Symbol' column
merged_df = pd.merge(company_df, stock_df, on='Symbol', how='inner')

# check the shape of the merged DataFrame
print(f"Merged DataFrame shape: {merged_df.shape}")

# check for missing values in the merged DataFrame
missing_merged = merged_df.isnull().any(axis=1).sum()
print(f"Number of rows in merged DataFrame with missing data: {missing_merged}")

# Remove duplicate rows
merged_df = merged_df.drop_duplicates()

Merged DataFrame shape: (229198, 21)
Number of rows in merged DataFrame with missing data: 0


In [6]:
def detect_format(date_str):
    # Try a list of common formats
    formats = [
        "%Y-%m-%d", "%d-%m-%Y", "%m-%d-%Y",
        "%d/%m/%Y", "%m/%d/%Y", "%Y/%m/%d",
        "%Y.%m.%d", "%d.%m.%Y", "%m.%d.%Y"
    ]
    for fmt in formats:
        try:
            datetime.strptime(date_str, fmt)
            return fmt
        except ValueError:
            continue
    return "Unknown format"

# Ensure all entries are strings
date_strings = merged_df['Date'].dropna()

# Apply format detection and count
format_counts = date_strings.apply(detect_format).value_counts()

# Display the result
print(format_counts)

Date
%d/%m/%Y    217318
Name: count, dtype: int64


In [7]:
merged_df['Date'] = pd.to_datetime(merged_df['Date'], format='mixed', dayfirst=True, errors='coerce')

merged_df = merged_df.sort_values(["Symbol", "Date"]) # Order by Symbol and Date 

lag_window = 10                             
for k in range(1, lag_window + 1):
    merged_df[f"Close_lag{k}"] = (
        merged_df.groupby("Symbol")["Close"]
          .shift(k)                          # k-day look-back
    )

lag_cols = [f"Close_lag{k}" for k in range(1, lag_window + 1)]
merged_df = merged_df.dropna(subset=lag_cols).reset_index(drop=True)

merged_df.head()

,Symbol,GICS Sector,Headquarters Location,Founded,Date,Close,Volume,News - Positive Sentiment,News - Negative Sentiment,News - New Products,...,Close_lag1,Close_lag2,Close_lag3,Close_lag4,Close_lag5,Close_lag6,Close_lag7,Close_lag8,Close_lag9,Close_lag10
0,A,Health Care,"Santa Clara, California",1999,2020-10-15,105.320000,723000,1.0,0.0,0.0,...,105.059998,105.419998,105.430000,105.760002,104.160004,103.879997,101.629997,103.120003,100.010002,101.220001
1,A,Health Care,"Santa Clara, California",1999,2020-10-16,106.699997,1039400,0.0,0.0,1.0,...,105.320000,105.059998,105.419998,105.430000,105.760002,104.160004,103.879997,101.629997,103.120003,100.010002
2,A,Health Care,"Santa Clara, California",1999,2020-10-19,105.489998,636000,0.0,0.0,1.0,...,106.699997,105.320000,105.059998,105.419998,105.430000,105.760002,104.160004,103.879997,101.629997,103.120003
3,A,Health Care,"Santa Clara, California",1999,2020-10-20,105.610001,771000,2.0,0.0,0.0,...,105.489998,106.699997,105.320000,105.059998,105.419998,105.430000,105.760002,104.160004,103.879997,101.629997
4,A,Health Care,"Santa Clara, California",1999,2020-10-21,104.830002,894000,0.0,0.0,0.0,...,105.610001,105.489998,106.699997,105.320000,105.059998,105.419998,105.430000,105.760002,104.160004,103.879997


In [8]:
# Change to classification problem
merged_df['Profit'] = (merged_df['Close'] > merged_df['Close_lag1']).astype(int)

merged_df['Profit'].value_counts()

Profit
1    109036
0    103332
Name: count, dtype: int64

In [9]:
# Save the merged DataFrame to a new CSV file
merged_df.to_csv('../Data/merged.csv', index=False)